In [ ]:
import pandas as pd
import numpy as np
import plotly.figure_factory as ff
from sklearn.metrics import confusion_matrix


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


from keras.utils import to_categorical
from keras import layers, Sequential
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import confusion_matrix
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [ ]:
# Replace 'filename.csv' with the path to your CSV file
df = pd.read_csv('Features_NN.csv')

In [ ]:
# Convert features into numpy array
X = df.drop(columns=['class','slice_file_name']).values

# Encode classes
encoder = LabelEncoder()
Y = encoder.fit_transform(df['class'])
Y = to_categorical(Y, num_classes=10)

In [ ]:
# Train-validation split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y,test_size=0.2, stratify=Y, random_state=18)

In [ ]:


model = Sequential([
    layers.Dense(512, activation='relu', input_shape=(138,)),
    layers.BatchNormalization(axis=-1),
    layers.Dropout(0.2),
    
    layers.Dense(512, activation='relu'),
    layers.BatchNormalization(axis=-1),
    layers.Dropout(0.2),
    
    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(axis=-1),
    layers.Dropout(0.2),
    
    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(axis=-1),
    layers.Dropout(0.2),
    
    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(axis=-1),
    layers.Dropout(0.2),
    
    layers.Dense(64, activation='relu'),
    layers.BatchNormalization(axis=-1),
    layers.Dropout(0.2),
    
    layers.Dense(10, activation='softmax')
])


In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Define callbacks
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    min_delta=5e-4,
    patience=10,
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_accuracy',
    factor=0.5,
    patience=3,
    min_lr=1e-5
)

In [ ]:
# Train the model
Train_model = model.fit(
    X_train, Y_train,
    validation_data=(X_test, Y_test),
    epochs=100,
    callbacks=[early_stopping, reduce_lr]
)

In [ ]:
# Compute confusion matrix
predicted = np.argmax(model.predict(X_test), axis=1)
true = np.argmax(Y_test, axis=1)
matrix = confusion_matrix(true, predicted)

# Plot confusion matrix
labels = list(encoder.classes_)

fig = ff.create_annotated_heatmap(z=matrix, x=labels, y=labels, colorscale='Viridis')
fig.update_layout(
                  xaxis=dict(title='Predicted Class'),
                  yaxis=dict(title='True Class'),
                  font=dict(size=12),
                  width=1000,
                  height=500)